In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
df = pd.read_csv("/content/nasa_power_prepared_data.csv", parse_dates=["date"])
df = df.sort_values("date")

# Target column
FEATURES = ["t2m", "rh2m", "ws2m", "et_total"]
data = df[FEATURES].values

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

In [ ]:
def create_sequences(dataset, time_steps=365):
    X, y = [], []
    for i in range(len(dataset)-time_steps):
        X.append(dataset[i:i+time_steps, 0])
        y.append(dataset[i+time_steps, 0])
    return np.array(X), np.array(y)

TIME_STEPS = 365  # ใช้ข้อมูลย้อนหลัง 365 วัน
X, y = create_sequences(scaled_data, TIME_STEPS)

# Reshape for LSTM [samples, time_steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(TIME_STEPS, 1)))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

In [ ]:
y_pred = model.predict(X_test)

# Inverse transform back to original scale
y_test_inv = scaler.inverse_transform(y_test.reshape(-1,1))
y_pred_inv = scaler.inverse_transform(y_pred)

In [ ]:
rmse = np.sqrt(np.mean((y_test_inv - y_pred_inv)**2))
print(f"LSTM RMSE: {rmse:.3f}")

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test_inv, label="Actual", color="blue")
plt.plot(y_pred_inv, label="LSTM Prediction", color="red", alpha=0.7)
plt.title("T2M Prediction with LSTM")
plt.xlabel("Time Steps")
plt.ylabel("Temperature (T2M)")
plt.legend()
plt.show()